Regression Model to Predict Program Performance Score

In [1]:
import numpy as np
import pandas as pd
import math

from sklearn import linear_model, preprocessing
from sklearn.neural_network import MLPRegressor

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')

/var/folders/w8/rx32_whx4n158n20x1kr4hph0000gn/T/ipykernel_4590/3732955502.py:11: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
x_data = pd.read_csv('x_all.csv')
x = x_data.to_numpy()
x_data

,total-operators,distinct-operators,total-operands,distinct-operands,function-calls,loops,assignments,max-cyclomatic-complexity,sum-cyclomatic-complexity,dynamic-memory-calls,file-access,file-operation,file-position,input-output,optimization
0,643,61,358,209,54,2,13,3,5,4,6,4,5,30,0
1,899,96,471,268,75,3,8,5,8,6,6,4,5,30,0
2,681,65,375,207,56,3,10,5,9,6,6,4,5,30,0
3,659,68,362,213,57,2,6,4,7,4,6,4,5,31,0
4,659,68,362,213,57,2,6,4,7,4,6,4,5,31,1
5,995,75,552,346,76,3,9,8,12,4,6,5,5,31,0
6,649,61,369,212,67,1,3,2,4,4,10,5,8,35,0
7,638,62,344,207,54,2,5,3,5,4,6,4,5,30,0
8,606,56,327,202,56,0,0,2,3,4,6,4,5,30,0
9,891,70,496,333,71,1,2,3,5,6,6,4,5,30,0


In [3]:
y_data = pd.read_csv('y_all.csv')
y = y_data.to_numpy()
y_data

,score
0,0.004178
1,0.068652
2,0.148324
3,0.051577
4,2.969427
5,1689.001461
6,14.685588
7,0.001842
8,0.006010
9,25.077806


In [6]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

ann = MLPRegressor(max_iter=2000, learning_rate_init=0.1, learning_rate='constant')
ann.fit(x_scaled[1:-1], y[1:-1].ravel())
print(ann.score(x_scaled[1:-1], y[1:-1]))

print(ann.predict([x_scaled[0]]))
print(y[0])
print(ann.predict([x_scaled[-1]]))
print(y[-1])


0.9999536255443205
[2.28287237]
[0.00417832]
[101.97271599]
[5.16688155]
